In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import torchvision as tv

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

from torch.cuda.amp import autocast, GradScaler
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
import pandas as pd

In [36]:
!git clone https://github.com/IvanDrokin/torch-conv-kan.git
%cd torch-conv-kan
from kan_convs import KANConv2DLayer

c:\ITMO_CAI\2023-2024\Wine_Aroma\torch-conv-kan\torch-conv-kan


Cloning into 'torch-conv-kan'...
c:\Users\lera-\.virtualenvs\-SNfpu_4o\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning:

using dhist requires you to install the `pickleshare` library.



In [39]:
X_array =  np.load('C:/ITMO_CAI/2023-2024/Wine_Aroma/X_array.npy')
Y_array =  np.load('C:/ITMO_CAI/2023-2024/Wine_Aroma/Y_array.npy')

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_array, Y_array, test_size=0.2, random_state=998)

In [41]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

In [42]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

359
359
90
90


In [43]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [44]:
# Создаем Dataset и DataLoader
dataset_train = CustomDataset(X_train, y_train)
dataset_test = CustomDataset(X_test, y_test)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=5, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=5, shuffle=False)

In [45]:
for x, y in dataloader_train:
    print(x.size())

torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])
torch.Size([5, 44, 100])


In [54]:
class SimpleConvKAN(nn.Module):
    def __init__(
            self,
            layer_sizes,
            num_classes: int = 10,
            input_channels: int = 1,
            spline_order: int = 3,
            groups: int = 1):
        super(SimpleConvKAN, self).__init__()

        self.layers = nn.Sequential(
            KANConv2DLayer(input_channels, 16, kernel_size=3, spline_order=spline_order, groups=1, padding=1, stride=1, dilation=1),
            KANConv2DLayer(16, 32, kernel_size=3, spline_order=spline_order, groups=groups, padding=1, stride=2, dilation=1),
            KANConv2DLayer(layer_sizes[1], layer_sizes[2], kernel_size=3, spline_order=spline_order, groups=groups, padding=1, stride=2, dilation=1),
            KANConv2DLayer(layer_sizes[2], layer_sizes[3], kernel_size=3, spline_order=spline_order, groups=groups, padding=1, stride=1, dilation=1),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.output = nn.Linear(layer_sizes[3], num_classes)

        self.drop = nn.Dropout(p=0.25)

    def forward(self, x):
        x = self.layers(x)
        x = torch.flatten(x, 1)
        x = self.drop(x)
        x = self.output(x)
        return x

# Создание экземпляра нейронной сети
net = SimpleConvKAN(layer_sizes=[16, 32, 64, 128])
# Печать архитектуры нейронной сети
print(net)

SimpleConvKAN(
  (layers): Sequential(
    (0): KANConv2DLayer(
      (base_activation): GELU(approximate='none')
      (base_conv): ModuleList(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (spline_conv): ModuleList(
        (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (layer_norm): ModuleList(
        (0): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (prelus): ModuleList(
        (0): PReLU(num_parameters=1)
      )
    )
    (1): KANConv2DLayer(
      (base_activation): GELU(approximate='none')
      (base_conv): ModuleList(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (spline_conv): ModuleList(
        (0): Conv2d(128, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (layer_norm): ModuleList(
        (0): InstanceNorm2d(32, eps=1e-

In [55]:
for param in net.parameters():
    param.data = param.data.double()

In [56]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer,
    gamma = 0.6
)

In [57]:
def accuracy(pred, label, threshold=0.5):
    pred = torch.sigmoid(pred)
    pred_labels = (pred > threshold).float()  # Threshold the predictions
    correct = (pred_labels == label).sum().item()  # Compare predictions with labels
    total = label.size(0) * label.size(1)  # Total number of labels
    return correct / total

In [58]:
device = 'cuda' # if torch.cuda.is_available() else 'cpu'
model = net.to(device)
loss_fn = loss_fn.to(device)

In [59]:
use_amp = True
scaler = torch.cuda.amp.GradScaler()
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

In [60]:
epochs = 50

for epoch in range(epochs):
    loss_val = 0.0
    acc_val = 0.0
    for sample in tqdm(dataloader_train):
        matrix, label = sample[0].to(device), sample[1].to(device)
        optimizer.zero_grad()

        # Добавим измерение каналов к данным
        matrix = matrix.unsqueeze(1)
  
        with autocast(use_amp):
            pred = model(matrix)
            loss = loss_fn(pred, label.float())

        scaler.scale(loss).backward()
        loss_item = loss.item()
        loss_val += loss_item

        scaler.step(optimizer)
        scaler.update()

        acc_current = accuracy(pred.cpu().float(), label.cpu().float())
        acc_val += acc_current

    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {loss_val/len(dataloader_train):.5f}, Accuracy: {acc_val/len(dataloader_train):.3f}')

100%|██████████| 72/72 [00:15<00:00,  4.61it/s]


Epoch: [1/50], Loss: 0.65916, Accuracy: 0.673


100%|██████████| 72/72 [00:11<00:00,  6.54it/s]


Epoch: [2/50], Loss: 0.60430, Accuracy: 0.768


100%|██████████| 72/72 [00:11<00:00,  6.52it/s]


...

Epoch: [48/50], Loss: 0.44146, Accuracy: 0.799


100%|██████████| 72/72 [00:18<00:00,  3.81it/s]


Epoch: [49/50], Loss: 0.43852, Accuracy: 0.801


100%|██████████| 72/72 [00:18<00:00,  3.83it/s]

Epoch: [50/50], Loss: 0.43644, Accuracy: 0.803


In [61]:
loss_val = 0.0
acc_val = 0.0
for sample in tqdm(dataloader_test):
    matrix, label = sample[0].to(device), sample[1].to(device)

    # Добавим измерение каналов к данным
    matrix = matrix.unsqueeze(1)
  
    with autocast(use_amp):
        pred = model(matrix)
        loss = loss_fn(pred, label)

    loss_item = loss.item()
    loss_val += loss_item


    acc_current = accuracy(pred.cpu().float(), label.cpu().float())
    acc_val += acc_current


print(f'Loss: {loss_val/len(dataloader_test):.5f}, Accuracy: {acc_val/len(dataloader_test):.3f}')

100%|██████████| 18/18 [00:02<00:00,  8.75it/s]

Loss: 0.43386, Accuracy: 0.803


In [62]:
def aroma_map(matrix):
    matrix = matrix.unsqueeze(1)
    pred = model(matrix).to('cpu')


    df = pd.DataFrame(dict(
        r=torch.sigmoid(pred).detach().numpy()[0],
        theta=['Herbs and spices', 'Tobacco/Smoke', 'Wood', 'Berries', 'Citrus',
       'Fruits ', 'Nuts', 'Coffee', 'Chocolate/Cacao', 'Flowers']))
    
    fig = px.line_polar(df, r='r', theta='theta', line_close=True, template="plotly_dark")
    fig.update_traces(fill='toself', line_color='maroon')
    fig.update_layout(polar=dict(radialaxis=dict(range=[0, 1])))
    fig.show()


In [63]:
aroma_map(X_train[42].to(device).unsqueeze(0))

In [64]:
def aroma_map_comparison(matrix, labels):
    matrix = matrix.unsqueeze(1)
    pred = model(matrix).to('cpu')
    labels = labels.to('cpu')

    print(labels)

    df = pd.DataFrame(dict(
        r=torch.sigmoid(pred).detach().numpy()[0],
        theta=['Herbs and spices', 'Tobacco/Smoke', 'Wood', 'Berries', 'Citrus',
       'Fruits ', 'Nuts', 'Coffee', 'Chocolate/Cacao', 'Flowers']))
    
    df['Label'] = 'Predict'
    
    df1 = pd.DataFrame(dict(
        r=labels.detach().numpy(),
        theta=['Herbs and spices', 'Tobacco/Smoke', 'Wood', 'Berries', 'Citrus',
       'Fruits ', 'Nuts', 'Coffee', 'Chocolate/Cacao', 'Flowers']))
    
    df1['Label'] = 'Experiment'
    
    fig = px.line_polar(pd.concat([df, df1]), color="Label",  r='r', theta='theta', line_close=True, line_shape='linear', color_discrete_sequence=['#008080', '#FFC0CB'],
                    template="plotly_dark")
    

    
    fig.update_traces(fill='toself')
    fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])))
    fig.show()

In [65]:
for i in range(90):
    matrix, label = X_test[i].to(device).unsqueeze(0), y_test[i].to(device)
    aroma_map_comparison(matrix, label)

tensor([0., 0., 0., 0., 1., 1., 0., 0., 0., 0.], dtype=torch.float64)


tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=torch.float64)


tensor([1., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)


tensor([1., 0., 1., 0., 0., 1., 0., 0., 0., 0.], dtype=torch.float64)


tensor([1., 0., 0., 0., 1., 1., 0., 0., 0., 0.], dtype=torch.float64)


...

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1.], dtype=torch.float64)


tensor([1., 0., 0., 1., 0., 1., 0., 0., 0., 0.], dtype=torch.float64)


tensor([1., 0., 0., 0., 1., 1., 0., 0., 0., 1.], dtype=torch.float64)


tensor([1., 0., 0., 1., 0., 0., 0., 1., 0., 1.], dtype=torch.float64)


tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.], dtype=torch.float64)
